<a id='Top'></a>
# UNSUPERVISED CHALLENGE

© Explore Data Science Academy

---

# Team JM4 Unsupervised Predict Notebook

---
<img src="img/predict bg.png" align="left">


<a id="cont"></a>

## Table of Contents

## <a href=#one>1. Introduction</a>
* <a href=#problemstament>1.1 Problem Statement</a>
* <a href=#po>1.2 Project Objectives</a>
* <a href=#dotrain_eda>1.3 Data Overview</a>
* <a href=#StartingaCometexperiment>1.4 Starting a Comet experiment</a>

## <a href=#two>2. Import Necessary Libraries</a>

## <a href=#three>3. Loading Datasets</a>
* <a href=#three1>3.1 Set Pandas to enable viewing of all columns</a>
* <a href=#three2>3.2 Check the "Shape" of the data-sets</a>
* <a href=#three21>3.3 Use the ".column" function to view the columns in our data set</a>

## <a href=#four>4. Data Preprocessing (Cleaning)</a>
* <a href=#four1>4.1 Identifying Missing Values</a>
* <a href=#four2>4.2 Preprocessing the Dataset</a>
<!-- * <a href=#four3>4.3 Most Common Words</a> -->

## <a href=#five>5. Exploratory Data Analysis (EDA)</a>



 <a id="one"></a>
## 1. INTRODUCTION
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Project Overview ⚡ |
| :--------------------------- |
| We will address the problem statement and objectives, as well as the classification of data aspects and a brief discussion of the Movie Recomender System in this part.|

<a id="top"></a>
- Recommender systems are economically and socially essential in today's technologically advanced world for enabling people to make wise decisions about the information they consume on a daily basis. This is particularly true in the area of movie content suggestions, where clever algorithms may guide viewers toward excellent films among tens of thousands of possibilities.

- We will build a recommendation system based on collaborative filtering or content that will be capable of effectively predicting how a user would rate a film they haven't yet seen based on their past preferences..

- With users of the system being exposed to content they would like to watch or buy, the provision of an accurate and robust solution to this problem has enormous economic potential, creating income and platform loyalty.
---



 <a id="problemstament"></a> 
 
### 1.1 Problem Statement

Our team has been charged with developing a collaborative filtering or content-based recommendation system that can precisely anticipate how a user would score a movie they haven't yet seen based on their prior preferences.

<a id="po"></a>

### 1.2 Project Objectives

* examine the data provided, find any probable flaws, and clean the current data set

* ascertain whether further characteristics may be included to improve the data set

* create a model that can forecast how a consumer would evaluate a film

* assess the best machine learning model's accuracy

* using a user's previous choices to forecast with accuracy how they would evaluate a movie they haven't yet seen

* Describe the model's inner workings to a non-technical audience.

<a id="dodf"></a>

### 1.3 Data Overview

Millions of users of the online MovieLens movie recommendation service provided their 5-star ratings for this dataset, which comprises of several million ratings. In order to enhance the functionality of explicitly-based recommender systems, business and academic researchers have long used the MovieLens dataset. 

We'll be utilizing an enhanced, resampled, and unique version of the Movies dataset for this Predict in order to conduct a fair evaluation.

#### I. Supplied Files
* genome_scores.csv: a rating indicating how well movies match certain tag-related qualities.

* genome_tags.csv: user-assigned tags for scores connected to the genome

* imdb_data.csv: Using the links.csv file, more movie metadata is extracted from IMDB.

* links.csv: File supplying a mapping between a related IMDB ID and TMDB ID and a Movies ID.

* tags.csv: For each movie in the collection, a user-assigned number.

* test.csv: The test split of the dataset. Contains user and movie IDs with no rating data.

* train.csv: The dataset's training segment. Contains user and movie IDs and the ratings that go with them.


>#### Additional Information
The below information is provided directly from the Movies dataset description files:

### Ratings Data File Structure (train.csv)
- The train.csv file contains all ratings. Following the header row, each line of this file contains one user's rating of one movie, in the following format:
    - userId
    - movieId
    - rating
    - timestamp

- Within this file, the lines are arranged by userId first, then within user, by movieId.

- A 5-star rating system is used, with half-star increments (0.5 stars - 5.0 stars).

- Timestamps are seconds since midnight on January 1, 1970, Coordinated Universal Time (UTC).

### Tags Data File Structure (tags.csv)
- The file tags.csv contains all of the tags. After the header row, each line in this file indicates a tag a user has applied to a particular video, and it is formatted as follows:
    - userId
    - movieId
    - tag
    - timestamp

- Within this file, the lines are arranged by userId first, then within user, by movieId.

- User-generated metadata about movies is created using tags. Typically, each tag consists of a single word or brief sentence. Each user decides what a given tag means, is worth, and serves for.

- Timestamps show seconds since midnight on January 1, 1970, Coordinated Universal Time (UTC).

### Movies Data File Structure (movies.csv)
- The movies.csv file contains movie-related data. Following the header row, each line in this file represents a single movie and is formatted as follows:
    - movieId
    - title
    - genres

- Genres are a pipe-separated list, and are selected from the following:
    - Action
    - Adventure
    - Animation
    - Children's
    - Comedy
    - Crime
    - Documentary
    - Drama
    - Fantasy
    - Film-Noir
    - Horror
    - Musical
    - Mystery
    - Romance
    - Sci-Fi
    - Thriller
    - War
    - Western
    (no genres listed)

### Links Data File Structure (links.csv)
- The file links.csv contains identifiers that may be used to link to various sources of movie data. This file's lines after the header row represent individual movies and are formatted as follows:
    - movieId
    - imdbId
    - tmdbId

- movieId is the movie identification that https://movies.org uses. For instance, https://movielens.org/movies/1 provides a link to the movie Toy Story.

- The movie identification used by http://www.imdb.com is called imdbId. Toy Story, for instance, may be found at http://www.imdb.com/title/tt0114709.

- tmdb

- Id is the movie identification that https://www.themoviedb.org uses. Toy Story, for instance, may be found at https://www.themoviedb.org/movie/862.

### Tag Genome (genome-scores.csv and genome-tags.csv)
- As previously mentioned, the tag genome encodes the degree to which movies display specific characteristics that are denoted by tags (atmospheric, thought-provoking, realistic, etc.). Using a machine learning algorithm on user-generated content, such as tags, ratings, and textual reviews, the tag genome was calculated.
- Two files make up the genome. Movie-tag relevance information is presented in the following way in the file genome-scores.csv:
    - movieId
    - tagId
    - relevance
- The tag descriptions for the tag IDs in the genome file are provided in the second file, genome-tags.csv, in the following format:
    - tagId
    - tag

<a id="StartingaCometexperiment"></a>

### 1.4 Starting a Comet Experiment
<img src="https://www.comet.ml/images/logo_comet_light.png" width="350" alt="Drawing" style="width: 350px;"/>

- Comet offers data scientists and teams a self-hosted and cloud-based meta machine learning platform that enables tracking, comparing, explanatory, and model optimization.

- Comet offers insights and data to construct stronger, more accurate AI models while enhancing productivity, collaboration, and visibility across teams. Comet is supported by thousands of users and several Fortune 100 firms.
- Comet will be used by us to version control our research.

In [ ]:
import os
os.environ["COMET_URL_OVERRIDE"] = "https://www.comet.com/clientlib/"
from comet_ml import Experiment

In [ ]:
# Creating an experiment with your api key
experiment = Experiment(
    api_key="bEFY9Hn1QccermEDT6aTyQMOA",
    project_name="jm4",
    workspace="kojosbk",
)

 <a id="two"></a>
## 2. Import Necessary Libraries
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Import necessary libraries ⚡ |
| :--------------------------- |
| We'd be importing all of the necessary libraries for the notebook to run smoothly..|



In [ ]:
# Libraries for importing and loading data
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import seaborn as sns
from textwrap import wrap
import pickle

#visualization libraries
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

# Entity featurization and similarity computation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Performance Evaluation
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

# Data Preprocessing
import random
from time import time
import cufflinks as cf
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.preprocessing import StandardScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Models
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, NMF, SlopeOne, CoClustering
from sklearn.metrics.pairwise import cosine_similarity


# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Setting global constants to ensure notebook results are reproducible

RANDOM_STATE = 42


import warnings
warnings.filterwarnings('ignore')


<a id="Three"></a>
## 3. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| The data from the `train` file is loaded into a DataFrame in this section.. |

---

#### - Train and test data:
For training and Kaggle submission purposes, this data is used. The target variable is rating.

In [ ]:
# load the data
#Note : the data souce is saved in another directory of the local machine  to avoid github Push issues
train = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/train.csv')
test = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/test.csv')

In [ ]:
# load the  Movie Data
#Note : the Movie Data data souce is saved in another directory of the local machine  to avoid github Push issues
genome_scores = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/genome_scores.csv', index_col='movieId')
genome_tags = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/genome_tags.csv')
imdb_data = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/imdb_data.csv')
links = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/links.csv')
movies = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/movies.csv')
tags = pd.read_csv('C:/Users/Silas_Dell/Documents/Programming/Explore/Unsupervised learning/data/tags.csv')

<a id=three1></a>

#### 3.1 Set Pandas to enable viewing of all columns
Due to the length of th content of the message column, pandas cannot display all of them at once by default. While doing EDA and data cleansing, we will need to see all of the columns. When the dataframe is presented, the code below allows us to see the whole set of columns in our data collection. 

In [ ]:
# Set option to display all columns
pd.set_option('display.max_colwidth', None)

<a id=three2></a>

#### 3.2 Check the "Shape" of the data-sets
As demonstrated by the shape of both datasets, the data has been separated into two sets. The form also shows that the training data set has four columns, but the test data set has just two. Our model is designed to forecast the column that is named movieID.

In [ ]:
# Preview train dataset
print('The Shape of the data is: ', train.shape)
train.head()

It should be noticed that the train data also includes a column called "timestamp." This information may be safely ignored because there is no connection between when someone watches a movie and whether or not they like it.

In [ ]:
# Preview test dataset
print('The Shape of the data is: ', test.shape)
test.head()

In [ ]:
# Preview genome_scores dataset
print('The Shape of the data is: ', genome_scores.shape)
genome_scores.head()

In [ ]:
# Preview genome_tags dataset
print('The Shape of the data is: ', genome_tags.shape)
genome_tags.head()

In [ ]:
# Preview imdb_data dataset
print('The Shape of the data is: ', imdb_data.shape)
imdb_data.head(2)

In [ ]:
# Preview links dataset
print('The Shape of the data is: ', links.shape)
links.head()

In [ ]:
# Preview movies dataset
print('The Shape of the data is: ', movies.shape)
movies.head()

In [ ]:
# Preview tags dataset
print('The Shape of the data is: ', tags.shape)
tags.head()

<a id="five"></a>
## 4. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In order to achieve the needed answers, uncover trends, patterns, and correlations that are not immediately obvious, or to get insights into the dataset, we will be analyzing and studying the data sets in this part. |

---

Exploratory data analysis is a way of studying data sets to highlight their key features, frequently using visual techniques. The crucial process of conducting early investigations on data using summary statistics and graphical representations to find trends, identify anomalies, test hypotheses, and verify assumptions.

EDA is primarily used to see what the data can tell us beyond the formal modeling or hypothesis testing work, regardless of whether a statistical model is utilized.

<a id=four1></a>

#### 5.1 Dataset summary
It is important to identify the columns that have null entries as null values can affect the performance of our model. The "isnull" function shows the number of null values that are contained in each column of the dataset. This data set is relatively clean 
Pandas dataframe.info() function is used to get a concise summary of the dataframe

In [ ]:
def Summary(df):
    i = df.info()
    print ("NUL Values")
    n = df.isna().sum()
    return i,n

In [ ]:
Summary(train)

In [ ]:
Summary(test)

In [ ]:
Summary(tags)

In [ ]:
Summary(movies)

In [ ]:
Summary(links)

In [ ]:
Summary(imdb_data)

In [ ]:
Summary(genome_tags)

In [ ]:
Summary(genome_scores)

#### Observations
The Dtype of int64 and float64, which is a sign of numeric values, is present in the majority of DataFrames. However, some DataFrames additionally feature an object's Dtype, which indicates a character other than a numeric character. The DataFrames tags df, movies df, imdb df, and genome df all have a Dtype.

### 5.2 Visualizing the dataframes

In [ ]:
# Created a Data Frame outlining the size of our data
dataframes = ['train', 'test', 'tags', 'imdb',
              'links', 'movies', 'genome_tags', 'genome_score']
sizes = [(train.shape[0]), (test.shape[0]), (tags.shape[0]),
         (imdb_data.shape[0]), (links.shape[0]), (movies.shape[0]),
         (genome_tags.shape[0]), (genome_scores.shape[0])]
total_size = pd.DataFrame(list(zip(dataframes, sizes)),
                             columns=['dataframe', 'sizes'])
total_size

In [ ]:
len_list = [['train', (train.shape[0])], ['tags', (tags.shape[0])],
            ['imdb', (imdb_data.shape[0])], ['links', (links.shape[0])],
            ['movies', (movies.shape[0])],
            ['genome_tags', (genome_tags.shape[0])],
            ['genome_score', (genome_scores.shape[0])]]
len = pd.DataFrame(len_list,
                      columns=['Dataset', 'Size'])
len = len.sort_values(by='Size', ascending=False)
fig, ax = plt.subplots( figsize=(12,6))
ax = sns.barplot(y =  len["Size"], x = len["Dataset"], palette='copper')
for p in ax.patches:
        ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % int(p.get_height()), fontsize=11, ha='center', va='bottom')
ax.set_xlabel("Data set Categories", fontsize = 15)
ax.set_ylabel("SIze of data set", fontsize = 15)
ax.set_title("Distribution of overall Data Frames", fontsize = 20)
plt.show()

In [ ]:
len_list = [['train', (train.shape[0])], ['tags', (tags.shape[0])],
            ['imdb', (imdb_data.shape[0])], ['links', (links.shape[0])],
            ['movies', (movies.shape[0])],
            ['genome_tags', (genome_tags.shape[0])],
            ['genome_score', (genome_scores.shape[0])]]
len = pd.DataFrame(len_list,
                      columns=['Dataset', 'Size'])
fig = px.treemap(len, title='Treemap chart by Distribution of overall Data Frames',
                 path=["Dataset"], values = 'Size',color='Size', color_continuous_scale=px.colors.sequential.GnBu)
fig.show()

#### Observations
The precise asymmetry in DataFrame size distribution is shown in the bar graph. The biggest DataFrame is the genome score DataFrame, followed by the train Dataframe. Whereas the bars in the other DataFrame are not readily visible due to the significant disparity in the dimensions, the difference in distribution sizes is obvious as shown in the treemap.

#### 5.2 Movies produced per Year
In this section, we're curious to learn which years had the greatest amount of films made. We may use this information to graphically examine historical performance of the film business.

In [ ]:
# Make a copy of movie table
movies_table = movies.copy()

# Remove delimiters from interested columns
movies_table["genres"] = movies_table["genres"].str.replace('|', ' ', regex=True)
movies_table["title"] = movies_table["title"].str.replace('(', ' ', regex=True)
movies_table["title"] = movies_table["title"].str.replace('$', ' ', regex=True)
movies_table["title"] = movies_table["title"].str.replace(')', ' ', regex=True)
movies_table["genres"] = movies_table["genres"].str.replace('(', ' ', regex=True)
movies_table["genres"] = movies_table["genres"].str.replace(')', ' ', regex=True)
# Create a movie year column
movies_table["year"] = movies_table["title"].str.replace(r'[a-zA-Z]', '', regex=True)
movies_table["year"] = movies_table["year"].str.replace(r' ', '', regex=True)
movies_table.head()

In [ ]:
# Visualise the first twenty years with the highest numbers of movies produced
movies_table["year"] = movies_table["year"].astype('str')
movies_year_count = movies_table["year"].value_counts()
fig, ax = plt.subplots( figsize=(12,6))
ax = sns.barplot(y =  movies_year_count.values[:20], x = movies_year_count.index[:20], palette='copper')
ax.set_xlabel("Years of Production", fontsize = 15)
ax.set_ylabel("Numbers of Movies Produced", fontsize = 15)
ax.set_title("First Twent Years with the Highest Numbers of Movies produced", fontsize = 20)
plt.show()

#### Quick Observations
* The year 2015 had the largest number of films created, with over 1700. It was followed by the years 2016 and 2017, which both saw over 1500 films produced.

* One thing to note is that a year like 2019 would have been expected to have a lot of movies released, but due to the outbreak of COVID 19, we observe a decline in movie releases in the year 2019.

#### 5.3 Top Rated Movies

For this area, we need to know which films have received a rating of 4 or above. Less than a 4 was seen to be an average rating.

In [ ]:
train_table = train.copy()
train_table.drop(columns='timestamp', inplace=True)
ratings = train_table[['movieId','rating']]
ratings = ratings[ratings['rating'] > 3.9]
ratings = ratings.merge(movies, on = 'movieId', how= 'left')

movies_ratings = ratings['title'].value_counts()

labels = [ '\n'.join(wrap(l, 30)) for l in movies_ratings.index[:20]]



fig, ax = plt.subplots( figsize=(12,6))
ax = sns.barplot(y =  movies_ratings.values[:20], x = labels, palette='copper')
ax.set_xlabel("Movie Title", fontsize = 20)
ax.set_ylabel("Numbers of Ratings", fontsize = 20)
ax.set_title("Top Twenty Rated Movies", fontsize = 30)
plt.xticks(rotation=90, fontsize= 7.7)
plt.show()


### key point to note

<p float="left">
  <img src="https://www.themoviedb.org/t/p/w500/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg" width="300" height = 400/>
  <img src="https://cdn.europosters.eu/image/750/posters/pulp-fiction-group-i1295.jpg" width="300" height = 400/>
  <img src="https://cps-static.rovicorp.com/2/Rights%20Managed/Belgacom/Forrest%20Gump/_derived_jpg_q90_310x470_m0/ForrestGump_EN.jpg" width="300" height = 400/>
</p>


**Top three Most Rated Movies**
- Shawshank Redemption (1994)
- Pulp Fiction (1994)
- Forrest Gump (1994)

### Directors

In [ ]:
def count_directors(df, count = 10):
    """
    The most frequent directors in a DataFrame may be counted using this function.:
    Parameters
    ----------
        df (DataFrame): dataframe with imdb metadata as input
        count (int): directors with fewer than count films to be filtered
        
    Returns
    -------
        directors (DataFrame): output DataFrame

    """
    directors = pd.DataFrame(df['director'].value_counts()).reset_index()
    directors.columns = ['director', 'count']
    # Lets only take directors who have made 10 or more movies otherwise we will have to analyze 11000 directors
    directors = directors[directors['count']>=count]
    return directors.sort_values('count', ascending = False)
directors = count_directors(imdb_data)

In [ ]:
plt.figure(figsize=(10,6))
drt = directors.copy()
# drt.drop(drt.index[0], inplace=True)
ax = sns.barplot(y = drt["director"].head(10), x = drt['count'], palette='copper', orient='h', edgecolor="black")
plt.title("Number of Movies Per director", fontsize=14)
plt.ylabel("director")
plt.xlabel('Count')
plt.show()

### Top 3 Directors with most movies released

<img height = "238" width = 178 src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/Luc_Besson_by_Gage_Skidmore.jpg/640px-Luc_Besson_by_Gage_Skidmore.jpg" alt="Photo of Luc Besson" class="GeneratedImage">  <img height = "238" width = 950 src="img\luc.JPG" alt="Movies of Tom Hanks" class="GeneratedImage">
</br>
<a href="https://en.wikipedia.org/wiki/Luc_Besson">Luc Paul Maurice Besson</a>  is a French filmmaker, writer, and producer of movies. The Big Blue, La Femme Nikita, and Subway were all movies he either directed or produced. </a>
</br>
</br>
<img height = "238" width = 178 src="https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQXYKDvhxIVt8R_yV3LLLZJ2LemcV860GqEgu9TKCDvGSDnHksM" alt="Photo of woody allen" class="GeneratedImage"> <img height = "238" width = 950 src="img\woody.JPG" alt="Movies of Tom Hanks" class="GeneratedImage">
</br>
<a href="https://en.wikipedia.org/wiki/Woody_Allen"> Woody Allen</a> is an American filmmaker, writer, actor, and comedian whose career spans more than six decades including several films that have won Academy Awards. </a>
</br>
</br>

<img height = "238" width = 178 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3ie6FvZbpJx2VbSMzbGsFagq2wPgnNXJxSPQVTI6ofqhWv28AZKCUZIt54kEQHr9gfiI&usqp=CAU" alt="Photo of Stephen King" class="GeneratedImage"> <img height = "238" width = 950 src="img\king.JPG" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Stephen_King">Stephen King</a>  is an American writer of books in the genres of horror, science fiction, fantasy, suspense, and paranormal fiction. </a>
</br>
</br>


---



#### Once again we need to calculate a mean rating for each director in order to determine who is the most popular

In [ ]:
def dir_mean(df):
    df.set_index('director', inplace=True)

    direct_ratings = []
    directors_eda = train.merge(imdb_data, how = 'left')
    for director in df.index:
        rating = round(directors_eda[directors_eda['director']==director]['rating'].mean(),2)
        direct_ratings.append(rating)
    df['mean_rating'] = direct_ratings
    return df.sort_values('mean_rating', ascending = False)
directors = dir_mean(drt)

In [ ]:
def feat_popularity(df, title = 'feat'):
    """
    Plots the mean rating per director.
    """
    plt.figure(figsize=(10,6))
    plot_data = df.copy()
    mean = plot_data['mean_rating'].mean()
    min_ = plot_data['mean_rating'].min()
    max_ = round(plot_data['mean_rating'].max(),2)
    sns.barplot(y = plot_data.index, x = plot_data['mean_rating'], order = plot_data.index, orient='h',palette='copper',edgecolor="black")
    plt.axvline(x=mean, label = f'mean {round(mean,1)}' , color='black', lw=1, ls ='--')
    plt.axvline(x=min_, label = f'min {round(min_,1)}' , color='#4D17A0', lw=1, ls = '--')
    plt.axvline(x=max_, label = f'max {max_}' , color='#4DA017', lw=1,ls = '--')
    plt.title(f'Mean Rating Per {title}', fontsize=14)
    plt.ylabel(f'{title}')
    plt.xlabel('Mean Rating')
    plt.legend(loc='lower center')
    plt.show()

In [ ]:
feat_popularity(directors.head(10), 'Director')

### Top 3 Directors with High ratings

<img height = "238" width = 178 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3ie6FvZbpJx2VbSMzbGsFagq2wPgnNXJxSPQVTI6ofqhWv28AZKCUZIt54kEQHr9gfiI&usqp=CAU" alt="Photo of Stephen King" class="GeneratedImage"> <img height = "238" width = 950 src="img\king.JPG" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Stephen_King">Stephen King</a>  is an American writer of books in the genres of horror, science fiction, fantasy, suspense, and paranormal fiction. </a>
</br>
</br>
<img height = "238" width = 178 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxlTrJvdxqSMBYf90USQe0qXEaMhXdy35FJOpUlEZ5PGl4wIBI" alt="Photo of Quentin Tarantino" class="GeneratedImage">  <img height = "238" width = 950 src="img\quent.JPG" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
</br>
<a href="https://en.wikipedia.org/wiki/Quentin_Tarantino"> Quentin Tarantino</a> is an American director, actor, critic, novelist, and screenwriter. His movies are known for their extensive allusions to pop culture and movie history, nonlinear plots, grim comedy, stylised violence, long dialogue, widespread use of profanity, cameos, and ensemble casts. </a>
</br>
</br>

<img height = "238" width = 178 src="https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcSFu6ohQVsNtOaLKFv6Qv3Xbp2GCIx54HXeTKy0qnVOiZEp4IFT" alt="Photo of John Sayles" class="GeneratedImage">  <img height = "238" width = 950 src="img\jon.png" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Stephen_King">John Sayles</a>   is an American independent film director, screenwriter, editor, actor, and novelist. He has twice been nominated for the Academy Award for Best Original Screenplay, for Passion Fish and Lone Star. His film Men with Guns was nominated for the Golden Globe for Best Foreign Language Film.</a>
</br>
</br>

### Key observatios
* From the list above, we can see certain directors who are immediately recognized. Stephen King and Quentin Tarantino are, predictably, at the top of the list.
* It comes as no surprise that the director of the film with the highest rating, Shawshank Redemption, is ranked first.
* The fact that the top 3 directors have an average mean rating of 4.0 further demonstrates how positively moviegoers rank their favorite films.
---



### Again, in order to identify which directors are the least well-liked in terms of ratings, we must compute the mean rating for each.

In [ ]:
feat_popularity(directors.tail(3), 'Director')

### Top 3 Directors with the lowest ratings

<img height = "125" width = 125 src="https://www.thepitchkc.com/content/uploads/2021/11/g/r/charles-band-scaled-e1637002282929.jpg" alt="Photo of Charles Band" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Charles_Band">Charles Band</a> is an American film producer and director, known for his work on horror comedy movies. </a>
</br>
</br>
<img height = "125" width = 125 src="https://upload.wikimedia.org/wikipedia/en/e/e3/John_Hughes_Home_Alone_2.jpg" alt="Photo of John Hughes" class="GeneratedImage"> 
</br>
<a href="https://en.wikipedia.org/wiki/John_Hughes_(filmmaker)"> John Hughes</a> was an American filmmaker. Hughes began his career in 1970 as an author of humorous essays and stories for the National Lampoon magazine. </a>
</br>
</br>

<img height = "125" width = 125 src="https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcQq8v6TWEuFEzwzeUmeVXsHWsR2nTst1wxrbHrSS77qnJMYtqog" alt="Photo of Clive Barker" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Clive_Barker">Clive Barker</a>   is an English playwright, author, film director and visual artist who came to prominence in the mid-1980s with a series of short stories, the Books of Blood, which established him as a leading horror writer. He has since written many novels and other works. </a>
</br>
</br>

### Key observatios
* From the list above, we can notice a few filmmakers that stand out right away. Puppet Master was created by Charles band, which gets very poor reviews.

* The least popular directors have an average mean rating of 2.5, which further exemplifies how poorly viewers evaluate their movies.
---



### Actors

In [ ]:
#this code takes over 97min to run for an i5 8th gen 12gig ram on a secon run it took
#67min.
"""
The most frequent Actors in a DataFrame may be counted using this code.:
"""
# Creat a dict to store values
im = imdb_data.copy()
im = im.dropna(axis=0)
cast_dict = {'title_cast': list(),
                'count': list(),}
# Retrieve a list of all possible cast
print('retrieving features...')
for imdb in range((im.shape[0])):
    cast = im['title_cast'].iloc[imdb].split('|')
    for cas in cast:
        if cas not in cast_dict['title_cast']:
            cast_dict['title_cast'].append(cas)
# count the number of occurences of each cast
print('counting...')
for castt in cast_dict['title_cast']:
    count = 0
    for imdb in range((im.shape[0])):
        cast = im['title_cast'].iloc[imdb].split('|')
        if castt in cast:
            count += 1
    cast_dict['count'].append(count)
    
    # Calculate metrics
cast = pd.DataFrame(cast_dict)
print('done!')
df = cast.sort_values(by = 'count', ascending=False)


### Visualizing the Actors per movie data

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.barplot(y = df['title_cast'].head(10), x = df['count'], palette='copper', orient='h',edgecolor="black")
plt.title('Number of Movies Per Actor For Top 10 Actors', fontsize=14)
plt.ylabel('Actors')
plt.xlabel('Number of movies')
plt.show()

### Top 3 Movie Actors with Highest Number of movies released

<img height = "238" width = 178 src="https://cdn.britannica.com/77/191077-050-63262B99/Samuel-L-Jackson.jpg" alt="Photo of Tom Hanks" class="GeneratedImage">
<img height = "238" width = 950 src="img\sam.JPG" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Tom_Hanks">Samuel Leroy Jackson </a>  is an American actor. One of the most widely recognized actors of his generation, the films in which he has appeared have collectively grossed over $27 billion worldwide, making him the highest-grossing actor of all time. </a>
</br>
</br>
<img height = "238" width = 178 src="https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcSItxlkc_a8e3O3T59cqB6Uw5iPRY5bJlmr8ZUt0KRPLObKcdTd" alt="Photo of Edward Norton" class="GeneratedImage">
<img height = "238" width = 950 src="img\wilss.jpg" alt="Movies of Bruce Willis" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Bruce_Willis">Bruce Willis</a>  is a famous American actor. In the 1970s, his acting career got its start on an off-Broadway theater. He rose to stardom in a starring role on the comedy-drama series Moonlighting (1985–1989), and he went on to feature in more movies, being known as an action hero for his performances as John McClane in the Die Hard trilogy (1988–2013) and other projects.</a>
</br>
</br>

<img height = "238" width = 178 src="https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcRpa6S_5nG_DBliLGNbMMqx_tSAxqmQqDbK26PsKIdZUPxZT017" alt="Photo of Steve Buscemi" class="GeneratedImage">
<img height = "238" width = 950 src="img\ste.jpg" alt="Movies of Leonardo DiCaprio" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Steve_Buscemi">Steve Buscemi</a> is an American actor and film producer. Known for his work as a leading man in biopics and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award, and three Golden Globe Awards.  </a>
</br>
</br>


#### Key point to consider
- Samule l. Jackson, Bruce wills, and Steve Buscemi are the top 3 actors with the most movies released.

- Samule l. Jackson has the most realsed movies at 69

---

### The popularity of the actors is not shown by the above chart, therefore let's compute a mean rating and add it to the data.

In [ ]:
def mean_calc(feat_df, ratings = train, movies = imdb_data, column = 'title_cast'):
    """
    Function to add mean ratings to a DataFrame:
    Parameters
    ----------
        feat_df (DataFrame): input dataframe containing cast data
        ratings (DataFrame): input dataframe containing ratings data
        movies  (DataFrame): input dataframe containing imdb metadata
        column  (object): input colum containing title cast
        
    Returns
    -------
        Mean ratings (DataFrame): output DataFrame

    """
    mov = movies.copy()
    mean_ratings = pd.DataFrame(ratings.merge(mov, how='left').groupby(['movieId'])['rating'].mean())
    movie_eda = movies.copy()
    movie_eda = movie_eda.join(mean_ratings, on = 'movieId', how = 'left')

    # Exclude missing values
    movie_eda = movie_eda
    movie_eda2 = movie_eda[movie_eda['rating'].notnull()]

    means = []
    for feat in feat_df[f'{column}']:
        mean = round(movie_eda2[movie_eda2[f'{column}'].str.contains(feat)]['rating'].mean(),2)
        means.append(mean)
    return means
cast['mean_rating'] = mean_calc(cast)
#sorting values via mean ratings
cast.sort_values('mean_rating', ascending=False)

In [ ]:
def genre_popularity(df):
    """
    Plots the mean rating per genre.
    Parameters
    ----------
        df (DataFrame): input dataframe containing cast data       
    Returns
    -------
        Graph (histogram): output histogram graph
    """
    count_filt = 20
    plt.figure(figsize=(10,6))
    plot_data = df[df['count']>count_filt].head(10)
    mean = plot_data['mean_rating'].mean()
    min_ = plot_data['mean_rating'].min()
    max_ = plot_data['mean_rating'].max()
    sns.barplot(y = plot_data['title_cast'], x = plot_data['mean_rating'], order = plot_data['title_cast'].head(10), orient='h',palette='copper')
    plt.axvline(x=mean, label = f'mean {round(mean,1)}' , color='blue', lw=1, ls ='--')
    plt.axvline(x=min_, label = f'min {round(min_,1)}' , color='#FF5D32', lw=1, ls = '--')
    plt.axvline(x=max_, label = f'max {max_}' , color='r', lw=1,ls = '--')
    plt.title(f'Mean Rating Per Actor (20+ movies released)', fontsize=14)
    plt.ylabel('Actors')
    plt.xlabel('Mean Rating')
    plt.legend(loc='lower center')
    plt.show()

In [ ]:
genre_popularity(cast.sort_values('mean_rating', ascending=False))

### Top 3 Movie Actors with High ratings (20+ movies released)

<img height = "238" width = 178 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS-bt7_ZKFMU8UOOjlq-GYF5P0iNJVuqz9HuDI3GkLmLXDfifpy" alt="Photo of Tom Hanks" class="GeneratedImage">
<img height = "238" width = 950 src="img\2tom_h.JPG" alt="Movies of Tom Hanks" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Tom_Hanks">Tom Hanks</a> is an American actor and filmmaker. Known for both his comedic and dramatic roles, he is one of the most popular and recognizable film stars worldwide, and is regarded as an American cultural icon. </a>
</br>
</br>
<img height = "238" width = 178 src="https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcTWkkwUYKcdK296LMj5-aAkieIrvt7pqfHc4N16BIvL9EHFSpuf" alt="Photo of Edward Norton" class="GeneratedImage">
<img height = "238" width = 950 src="img\tempsnip.png" alt="Movies of Edward Norton" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Edward_Norton">Edward Norton</a> is an American actor and filmmaker. He has received numerous awards and nominations, including a Golden Globe Award and three Academy Award nominations. Born in Boston, Massachusetts and raised in Columbia, Maryland, Norton was drawn to theatrical productions at local venues as a child. </a>
</br>
</br>

<img height = "238" width = 178 src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2q9tvih6sHPAEEbPoCRrWpf2IWVG5IOo5jIxqCA7dgrggsQO5" alt="Photo of Leonardo DiCaprio" class="GeneratedImage">
<img height = "238" width = 950 src="img\leo.jpg" alt="Movies of Leonardo DiCaprio" class="GeneratedImage"></br>
<a href="https://en.wikipedia.org/wiki/Leonardo_DiCaprio">Leonardo DiCaprio</a> is an American actor and film producer. Known for his work as a leading man in biopics and period films, he is the recipient of numerous accolades, including an Academy Award, a British Academy Film Award, and three Golden Globe Awards.  </a>
</br>
</br>

### Key observatios
* As we can see from the list above, practically every actor is easily identifiable. Naturally, Tom Hanks and Leonardo DiCaprio are at the top of the list.

* The average mean rating of 3.6 for the top 3 actors further indicates how highly moviegoers regard their favorite actors.
---



#### Consumer perception of movie reviews or ratings

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.distplot(train['rating'],bins=10, kde=False, hist_kws=dict(alpha=0.6),color="#FF5D32")
mean = train['rating'].mean()
median = train['rating'].median()
plt.axvline(x=mean, label = f'mean {round(mean,2)}' , color='#FF5D32', lw=3, ls = '--')
plt.axvline(x=median, label = f'median {median}' , color='#4DA017', lw=3, ls = '--')
plt.xlim((0.5,5))
plt.ylim((0,2500000))
plt.title(f'Distribution of Ratings', fontsize=14)
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.legend()
plt.show()

#### Key point to note

 - The ratings' left-leaning bias is intriguing. An average distribution with a mean rating of 3.5 was anticipated. Instead, we see that viewers often give movies positive reviews and steer clear of giving them bad ones. The inclination of users to rank movies they like may help to explain this bias. In other words, it is unlikely that a person will finish watching and rating a film if they don't enjoy it.
---

### relationship between the number of ratings a movie has and how highly it is rated

In [ ]:
plt.figure(figsize=(10,6))
mean_ratings = train.groupby("movieId")['rating'].mean()
user_counts = train.groupby("movieId")['movieId'].count().values
sns.scatterplot(x=mean_ratings, y = user_counts, color="#FF5D32", )
plt.title(f'Correlation between a movies rating and its rating count', fontsize=14)
plt.xlabel('Rating')
plt.ylabel('Number of Ratings')
plt.show()

According to the scatter plot shown above, a movie is more likely to receive a high rating the more ratings it has. This supports our innate belief that moviegoers are more inclined to suggest a film to one another if it has a higher rating. In other words, most individuals aim to avoid making unfavorable suggestions.

### Percentage distribution of users per rating

In [ ]:
movieRatingDistGroup = train['rating'].value_counts().sort_index().reset_index()
fig, ax = plt.subplots(figsize=(10,6))
sns.barplot(data=movieRatingDistGroup, x='index', y='rating', palette="copper", edgecolor="black", ax=ax)
ax.set_xlabel("Rating")
ax.set_ylabel('Number of Users')
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
total = float(movieRatingDistGroup['rating'].sum())
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height+350, '{0:.2%}'.format(height/total), fontsize=11, ha="center", va='bottom')
plt.title('Number of Users Per Rating', fontsize=14)
plt.show()

We can see from the bar graph up top that the majority of movies receive a rating of 4.0.

#### 5.4 Top Movie Viewers
We're interested in finding out which userIds watch the most movies at this point. This information may be used to determine the top clients and their preferred movies.

In [ ]:
# Top movie viewers
train_table['userId'] = train_table['userId'].astype('str')
viewers = train_table['userId'].value_counts()
fig, ax = plt.subplots( figsize=(10,6))

ax = sns.barplot(y = viewers.values[:20], x = viewers.index[:20], palette='copper',edgecolor="black")
ax.set_xlabel("UserIds", fontsize = 20)
ax.set_ylabel("Numbers of Movies", fontsize = 20)
ax.set_title("Top Twenty Movie Reviewers", fontsize = 30)
plt.xticks(rotation=90, fontsize= 10)
plt.yticks( fontsize= 10)
plt.show()

**Top Movie Viewers by user ID**
- 72315
- 80974
- 137293


### Genres

In [ ]:
# Creat a dict to store values
movies = movies.dropna(axis=0)
genre_dict = {'genres': list(),
                'count': list(),}
# Retrieve a list of all possible genres
print('retrieving features...')
for movie in range((movies.shape[0])):
    gens = movies['genres'].iloc[movie].split('|')
    for gen in gens:
        if gen not in genre_dict['genres']:
            genre_dict['genres'].append(gen)
# count the number of occurences of each genre
print('counting...')
for genre in genre_dict['genres']:
    count = 0
    for movie in range((movies.shape[0])):
        gens = movies['genres'].iloc[movie].split('|')
        if genre in gens:
            count += 1
    genre_dict['count'].append(count)
    
    # Calculate metrics
genres = pd.DataFrame(genre_dict)
print('done!')
df = genres.sort_values(by = 'count', ascending=False)
plt.figure(figsize=(10,6))
ax = sns.barplot(y = df['genres'], x = df['count'], palette='copper', orient='h',edgecolor="black")
plt.title('Number of Movies Per genres', fontsize=14)
plt.ylabel('genres')
plt.xlabel('Count')
plt.show()

#### Key point to consider
- Drama, comedy, and thriller are the top 3 movie genres in popularity.
Character, story, storyline, and setting are the four constituent aspects or pieces of a genre. Additionally, it's common to hear individuals say that a certain movie had a solid storyline or an interesting tale. in reality the characters, the issues/conflicts the characters encountered, and the resolution of those issues and conflicts are what people are really referring to. Drama may be the most popular genre since it focuses on character development in the story, frequently overcoming a variety of obstacles and conflicts, or human challenges.

- Drama provides the emotional and interpersonal growth of realistic individuals in a realistic context, according to Hayley Mckenzie. It delivers an openly depicted tale of human adversity and provides rich character development. And this may be the main factor behind the popularity of the drama genre.

- Approximately 5000 movies have missing genres.

- We can use the IMDB and TMDB ID's together with the APIs to fill missing data. Further, IMAX is not a genre but rather a proprietary system for mass-viewings.
---

### The above figure does not tell us anything about the popularity of the genres, lets calculate a mean rating and append it to the data

In [ ]:
def mean_calc(feat_df, ratings = train, movies = movies, metadata = imdb_data, column = 'genres'):
    mean_ratings = pd.DataFrame(ratings.merge(movies, how='left').groupby(['movieId'])['rating'].mean())
    movie_eda = movies.copy()
    movie_eda = movie_eda.join(mean_ratings, on = 'movieId', how = 'left')

    # Exclude missing values
    movie_eda = movie_eda
    movie_eda2 = movie_eda[movie_eda['rating'].notnull()]

    means = []
    for feat in feat_df[f'{column}']:
        mean = round(movie_eda2[movie_eda2[f'{column}'].str.contains(feat)]['rating'].mean(),2)
        means.append(mean)
    return means
genres['mean_rating'] = mean_calc(genres)
genres.sort_values('mean_rating', ascending=False).head(5)

In [ ]:
def genre_popularity(df):
    """
    Plots the mean rating per genre.
    """
    count_filt = 500
    plt.figure(figsize=(10,6))
    plot_data = df[df['count']>count_filt]
    mean = plot_data['mean_rating'].mean()
    min_ = plot_data['mean_rating'].min()
    max_ = plot_data['mean_rating'].max()
    sns.barplot(y = plot_data['genres'], x = plot_data['mean_rating'], order = plot_data['genres'], orient='h',palette='copper')
    plt.axvline(x=mean, label = f'mean {round(mean,1)}' , color='blue', lw=1, ls ='--')
    plt.axvline(x=min_, label = f'min {round(min_,1)}' , color='#FF5D32', lw=1, ls = '--')
    plt.axvline(x=max_, label = f'max {max_}' , color='r', lw=1,ls = '--')
    plt.title(f'Mean Rating Per Genre', fontsize=14)
    plt.ylabel('Genre')
    plt.xlabel('Mean Rating')
    plt.legend(loc='lower center')
    plt.show()

In [ ]:
genre_popularity(genres.sort_values('mean_rating', ascending=False))

#### Observations
<img height = "210" width = 400 src="https://imgix.bustle.com/fatherly/2019/06/best-nature-documentaries.jpg?w=1200&h=630&fit=crop&crop=faces&fm=jpg" align="center" alt="Photo of Clive Barker" class="GeneratedImage"></br>

- Documentries seams to be the most higly rated releases in the data

- The ratings are almost evenly distributed, with the exception of documentaries, conflict, drama, musicals, and romance, which score over average. On the other side, the ratings for thriller, action, science fiction, and horror are noticeably below average.

- Hollywood crime dramas are referred to as "film-noir," especially those that stress cynical attitudes and sexual desires. In general, the "classic era" of American film-noir is thought to have been the 1940s and 1950s. Though it's possible that their particular audience is why some films earn the greatest ratings. For IMAX movies, the same reasoning holds true; hence, we only accounted for categories with a count of 500 or more in this graph.

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in tags['tag']:

    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()

    # Converts each token into lowercase
    for i in range((np.shape(tokens)[0])):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width=1200, height=900,
                      colormap='copper',
                      background_color='white',
                      stopwords=stopwords,collocations=False,
                      min_font_size=10).generate(comment_words)

# plot the WordCloud image
plt.figure(figsize=(14, 7), facecolor="white")
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Distribution of words in the tags data frame by Tags\n----------------------------------------------------',fontsize=25,y=1.05 ,loc='center')
plt.tight_layout(pad=0)

plt.show()


<a id="four"></a>
## 6. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |


---

**Data(Feature) Engineering** is the process of using domain knowledge to reconfigure the data and create “features” that optimize machine learning algorithms. With the insight gained from the **EDA** section above, we have decided to work with the below listed table:
- train
- test
- movies
- imdb_data

Also, for this phase we shall execute the below task on our dataset:
- Identify the unique userIds in the test data and only work with same for the purpose of **Content-based** algorithm
- Sort both the train and test dataset by userId for both algorithms
- Merge the movie and imdb_data tables separately with both the test and train dataset, for both algorithms 
- Drop unwanted columns after the merge
- Generate a key word column, which is a collection of all vital features of a movie for **conten-based** algorithm
- Divide the dataset set into chunks of 162,350( unique userId per chunk), to ease data processing acivities like vectorization and evaluation of cosine_similarity

### Multidimensional Scaling
Multidimensional scaling (MDS) is a technique for visualizing distances between objects on a map, where the distance is known between pairs of the objects.  
> "The t-SNE algorithm comprises two main stages. First, t-SNE constructs a probability distribution over pairs of high-dimensional objects in such a way that similar objects have a high probability of being picked, whilst dissimilar points have an extremely small probability of being picked. Second, t-SNE defines a similar probability distribution over the points in the low-dimensional map, and it minimizes the [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between the two distributions with respect to the locations of the points in the map. Note that whilst the original algorithm uses the Euclidean distance between objects as the base of its similarity metric, this should be changed as appropriate." [[3]](#ref3)

In [ ]:
# Subset the data to cut down computation time for now
genome_score = genome_scores[:10000000]

In [ ]:
# Although scores are in the range of 0-1, there is no harm in scaling
scaler_mds = StandardScaler()
mds_genome = scaler_mds.fit_transform(genome_score.sample(frac=0.0001))

In [ ]:
tsne = TSNE(3, n_jobs = -1, verbose = 2, perplexity = 10, learning_rate = 0.1)

In [ ]:
tsne.fit(mds_genome)

In [ ]:
Axes3D

fig = plt.figure(figsize=(15, 8))

# Add 3D scatter plot
ax = fig.add_subplot(projection='3d')
ax.scatter(tsne.embedding_[:,0], tsne.embedding_[:,1], tsne.embedding_[:,2], color='#4D17A0')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
sns.scatterplot(x = tsne.embedding_[:,0], y = tsne.embedding_[:,1], size=tsne.embedding_[:,2],color='#4DA017')
plt.show()

### Principal Component Analysis
Principal component analysis (PCA) is a technique for reducing the dimensionality of such datasets, increasing interpretability but at the same time minimizing information loss. It does so by creating new uncorrelated variables that successively maximize variance. [[4]](#ref4)

In [ ]:
# Manually pivot table as data is too large for in-built functions
def pivot_(df):
    """
    Pivots table.
    """
    new_dict = {'movieId':sorted(set(df.index))}
    pivoted = pd.DataFrame(new_dict)
    tagids = sorted(set(df['tagId']))
    for Id in range(np.shape(tagids)[0]):
        pivoted[f'{Id+1}'] = list(df[df['tagId'] == Id+1]['relevance'])
    return pivoted

In [ ]:

pca_data_pivoted = pivot_(genome_scores).set_index('movieId')

In [ ]:
pca_data_pivoted.shape

Lets make this more readable

In [ ]:
pca_data_pivoted.columns = list(genome_tags['tag'])

In [ ]:
pca_data_pivoted.head()

In [ ]:
features = [col for col in pca_data_pivoted.columns]

#### Scales of measurement
It is important that we scale the data before dimensionality reduction.

Although all variables are measured on the same scale (0-1), there shouldn't be any downside to setting the mean to zero and standard deviation to 1

In [ ]:
cf.set_config_file(offline=True, world_readable=True, theme='white')
columns = random.sample(range(0, 1129), 20)
pca_data_pivoted.iloc[:,columns].iplot(kind='box', title="Boxplots of Features (Unscaled)")

In [ ]:
def scaler(df):
    """
    Scales data.
    """
    scaler = StandardScaler(with_std=True)
    scaled_data = scaler.fit_transform(df)
    return scaled_data

In [ ]:
pca_scaled = scaler(pca_data_pivoted)

In [ ]:
scaled_pca = pd.DataFrame(pca_scaled, index = pca_data_pivoted.index, columns = pca_data_pivoted.columns)

In [ ]:
cf.set_config_file(offline=True, world_readable=True, theme='white')
# using plotly to plot the boxplot
scaled_pca.iloc[:,columns].iplot(kind='box', title="Boxplots of Features (Scaled)")

In [ ]:
# define PCA object
pca = PCA()

# fit the PCA model to our data and apply the dimensionality reduction 
prin_comp = pca.fit_transform(pca_data_pivoted[features])

# create a dataframe containing the principal components
pca_df = pd.DataFrame(data = prin_comp,
                      index=pca_data_pivoted.index, columns=pca_data_pivoted.columns
                     )

# plot line graph of cumulative variance explained
plt.plot(np.cumsum(pca.explained_variance_ratio_),color='#4D17A0')
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

In [ ]:
pca_75 = PCA(.80)
pca_75_df = pca_75.fit_transform(pca_data_pivoted)
print(round(pca_75.explained_variance_ratio_.sum()*100, 1),
      "% of variance explained by",
      pca_75.n_components_,
      "components.")

The cummulative explained variance shows an initially steep then gradual curve and not the sharp elbow we were expecting. This could be a result of the genomes already having been chosen as the principle components of movies. However we can see that 80% of the variance in the movie dataset is explained by 131 components. We should use only these components for computatonal efficiency in a content based model

In [ ]:
pca_75_df = pd.DataFrame(pca_75_df, index = pca_data_pivoted.index)
pca_75_df.head()

### Cluster Analysis

In [ ]:
# Manually implement the WCSS
def within_cluster_variation(df, label_col='cluster_label'):
    """
    Manually implements the WCSS.
    """
    centroids = df.groupby(label_col).mean()
    out = 0
    for label, point in centroids.iterrows():
        df_features = df[df[label_col] == label].drop(label_col, axis=1)
        out += (df_features - point).pow(2).sum(axis=1).sum()
    return out

In [ ]:
# let's try everything between 2 and 18 clusters where 18 is the number of genres
n_clusters = np.arange(2, 19)

# store errors for each value of k
errors = []

# for i between 2 and 19
for k in n_clusters:
    print(f'training model with {k} clusters')
    # perform k-means clustering
    km = KMeans(n_clusters=k, n_init=10, max_iter=300, random_state=42)
    km.fit(pca_75_df)
    
    # measure BCSS
    print(f'evaluating model with {k} clusters')
    y_preds = km.predict(pca_75_df)
    pca_75_df = pd.DataFrame(pca_75_df)
    pca_75_df['cluster_label'] = y_preds
    errors.append(within_cluster_variation(pca_75_df, 'cluster_label'))
    print(errors[-1])

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method for Determining Optimal Value of k')
plt.scatter(n_clusters, errors, color="#4DA017")
plt.plot(n_clusters, errors)
plt.xticks(n_clusters)
plt.axvline(x=3, color='#4D17A0', lw=2)
plt.show()

In [ ]:
K = 3
# Remember to set the random state for reproducibility
km = KMeans(n_clusters=K, verbose=0, random_state=42)
print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(pca_75_df)
print("done in %0.3fs" % (time() - t0))

In [ ]:
# Obtain cluster memberships for each item in the data
y_preds = km.predict(pca_75_df)
pca_75_df['cluster_label'] = y_preds
centers = km.cluster_centers_

In [ ]:
plt.figure(dpi=120)
for k in range(K):
    x1 = pca_75_df[pca_75_df['cluster_label'] == k][0]
    x2 = pca_75_df[pca_75_df['cluster_label'] == k][1]
    plt.scatter(x1, x2, label="k = "+str(k+1),alpha=0.75)
# Show cluster centroid locations    
plt.scatter(centers[:,0],centers[:,1],label="centroid")
plt.legend()
plt.title(f"K = {K}")
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

<a id="modelling"></a>
## 5. Modelling

To reduce computation time, we train and evaluate the following models on a 100k subset of the data. The best performing model will be trained on the whole dataset to predict the ratings for the final submission file.

In [ ]:
# Load the 100k dataset
train.drop('timestamp', axis=1, inplace=True)
train_subset = train[:100000]
reader = Reader(rating_scale=(train_subset['rating'].min(), train_subset['rating'].max()))
data = Dataset.load_from_df(train_subset[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

### Collaborative Filtering

Collaborative filtering is a technique that can filter out items that a user might like on the basis of reactions by similar users. It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user [[5]](#ref5).

#### SVD

The Singular Value Decomposition algorithm is a matrix factorization technique which reduces the number of features of a dataset and was popularized by Simon Funk during the [Neflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize) contest [[6]](#ref6). In the matrix structure, each row represents a user and each column represents a movie. The matrix elements are ratings that are given to movies by users.

In [ ]:
svd_test = SVD(n_epochs = 30, n_factors = 200, init_std_dev = 0.05, random_state=42)
svd_test.fit(trainset)
predictions = svd_test.test(testset)
# Calculate RMSE
svd_rmse = accuracy.rmse(predictions)

#### NormalPredictor  
The Normal Predictor algorithm predicts a random rating for each movie based on the distribution of the training set, which is assumed to be normal.

In [ ]:
np_test = NormalPredictor()
np_test.fit(trainset)
predictions = np_test.test(testset)
# Calculate RMSE
np_rmse = accuracy.rmse(predictions)

#### BaselineOnly  
The Baseline Only algorithm predicts the baseline estimate for a given user and movie. A baseline is calculated using either Stochastic Gradient Descent (SGD) or Alternating Least Squares (ALS).

In [ ]:
bsl_options = {'method': 'sgd','n_epochs': 40}
blo_test = BaselineOnly(bsl_options=bsl_options)
blo_test.fit(trainset)
predictions = blo_test.test(testset)
# Calculate RMSE
blo_rmse = accuracy.rmse(predictions)

#### NMF  
NMF is a collaborative filtering algorithm based on Non-negative Matrix Factorization. The optimization procedure is a (regularized) stochastic gradient descent with a specific choice of step size that ensures non-negativity of factors, provided that their initial values are also positive.

In [ ]:
nmf_test = NMF()
nmf_test.fit(trainset)
predictions = nmf_test.test(testset)
# Calculate RMSE
nmf_rmse = accuracy.rmse(predictions)

#### SlopeOne  
The SlopeOne algorithm is a simple yet accurate collaborative filtering algorithm that uses a simple linear regression model to solve the data sparisity problem. 

In [ ]:
slo_test = SlopeOne()
slo_test.fit(trainset)
predictions = slo_test.test(testset)
# Calculate RMSE
slo_rmse = accuracy.rmse(predictions)

#### CoClustering  
The Co-clustering algorithm assigns clusters using a straightforward optimization method, much like k-means.

In [ ]:
cc_test = CoClustering(random_state=42)
cc_test.fit(trainset)
predictions = cc_test.test(testset)
# Calculate RMSE
cc_rmse = accuracy.rmse(predictions)

### Content-based Filtering  
Content-based filtering, also referred to as cognitive filtering, recommends items based on a comparison between the content of the items and a user profile. The content of each item is represented as a set of descriptors or terms, typically the words that occur in a document [[7]](#ref7). In the following section, the model uses genres as keywords to recommend similar movies based on input from a user. The model was not used to predict ratings for the testing data, as it is too computationally expensive.

In [ ]:
def data_preprocessing(subset_size):
    """Prepare data for use within Content filtering algorithm.

    Parameters
    ----------
    subset_size : int
        Number of movies to use within the algorithm.

    Returns
    -------
    Pandas Dataframe
        Subset of movies selected for content-based filtering.

    """
    # Split genre data into individual words.
    movies['keyWords'] = movies['genres'].str.replace('|', ' ')
    # Subset of the data
    movies_subset = movies[:subset_size]
    return movies_subset
 
def content_model(movie_list,top_n=10): 
    """Performs Content filtering based upon a list of movies supplied
       by the app user.

    Parameters
    ----------
    movie_list : list (str)
        Favorite movies chosen by the app user.
    top_n : type
        Number of top recommendations to return to the user.

    Returns
    -------
    list (str)
        Titles of the top-n movie recommendations to the user.

    """
    # Initializing the empty list of recommended movies
    data = data_preprocessing(2000)
    # Instantiating and generating the count matrix
    count_vec = CountVectorizer()
    count_matrix = count_vec.fit_transform(data['keyWords'])
    indices = pd.Series(data['title'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    cosine_sim = pd.DataFrame(cosine_sim, index = data.index, columns = data.index)
    # Getting the index of the movie that matches the title
    idx_1 = indices[indices == movie_list[0]].index[0]
    idx_2 = indices[indices == movie_list[1]].index[0]
    idx_3 = indices[indices == movie_list[2]].index[0]
    # Creating a Series with the similarity scores in descending order
    rank_1 = cosine_sim[idx_1]
    rank_2 = cosine_sim[idx_2]
    rank_3 = cosine_sim[idx_3]
    # Calculating the scores
    score_series_1 = pd.Series(rank_1).sort_values(ascending = False)
    score_series_2 = pd.Series(rank_2).sort_values(ascending = False)
    score_series_3 = pd.Series(rank_3).sort_values(ascending = False)
    # Getting the indexes of the 10 most similar movies
    listings = score_series_1.append(score_series_2).append(score_series_3).sort_values(ascending = False)
    # Store movie names
    recommended_movies = []
    # Appending the names of movies
    top_50_indexes = list(listings.iloc[1:50].index)
    # Removing chosen movies
    top_indexes = np.setdiff1d(top_50_indexes,[idx_1,idx_2,idx_3])
    for i in top_indexes[:top_n]:
        recommended_movies.append(list(movies['title'])[i])
    return recommended_movies

In [ ]:
movies = movies.dropna()
movie_list = ['Grumpier Old Men (1995)','Ace Ventura: When Nature Calls (1995)','Father of the Bride Part II (1995)']
content_model(movie_list,top_n=10)

<a id="evaluation"></a>
## 6. Performance Evaluation

We built and tested six different collaborative filtering models and compared their performance using a statistical measure known as the root mean squared error (**RMSE**), which determines the average squared difference between the estimated values and the actual value. A low RMSE value indicates high model accuracy.

### Root Mean Squared Error (RMSE):
$$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$   

In [ ]:
# Compare RMSE values between models
fig,axis = plt.subplots(figsize=(8, 5))
rmse_x = ['SVD','NormalPredictor','BaselineOnly','NMF','SlopeOne','CoClustering']
rmse_y = [svd_rmse,np_rmse,blo_rmse,nmf_rmse,slo_rmse,cc_rmse]
ax = sns.barplot(x=rmse_x, y=rmse_y,palette='brg',edgecolor='black')
plt.title('RMSE Value Per Collaborative-based Filtering Model',fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('RMSE')
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2, p.get_y() + p.get_height(), round(p.get_height(),2), fontsize=12, ha="center", va='bottom')
    
plt.show()

### Cross Validation

Cross validation is a technique used to test the accuracy of a model's prediction on unseen data (validation sets). This is important because it can assist in picking up issues such as over/underfitting and selection bias. We used the K-fold technique to perform cross validation on our two best perfoming models, i.e. **SVD** and **BaselineOnly**.

**SVD**

In [ ]:
svd_test = SVD(n_epochs = 40, n_factors = 200, init_std_dev = 0.05, random_state=42)
# Run 5-fold cross-validation and print results
a = cross_validate(svd_test, data, measures=['RMSE'], cv=5, verbose=True)

**BaselineOnly**

In [ ]:
bsl_options = {'method': 'sgd','n_epochs': 40}
blo_test = BaselineOnly(bsl_options=bsl_options)
# Run 5-fold cross-validation and print results
b = cross_validate(blo_test, data, measures=['RMSE'], cv=5, verbose=True)

<a id="analysis"></a>
## 7. Model Analysis

In [ ]:
# Load the 100k dataset
train_subset = train[:100000]
reader = Reader(rating_scale=(train_subset['rating'].min(), train_subset['rating'].max()))
data = Dataset.load_from_df(train_subset[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

### Hyperparameter Tuning 

Hyperparameter tuning is the process by which a set of ideal hyperparameters are chosen for a model. A hyperparameter is a parameter for which the value is set manually and tuned to control the algorithm's learning process. We tested multiple parameters for our best performing model (i.e. **SVD**).

In [ ]:
param_grid = {'n_epochs':[40], #[30,40,50],
              'n_factors':[400], #[100,200,300,400],
              'init_std_dev':[0.005], #[0.001,0.005,0.05,0.1],
              'random_state':[42]} 
grid_SVD = GridSearchCV(SVD, cv=5, measures=['rmse'], param_grid=param_grid, n_jobs=-1)
grid_SVD.fit(data)
print('***Best score:***')
print(grid_SVD.best_score['rmse'])
print('***Best parameters:***')
print(grid_SVD.best_params['rmse'])

### Results

In [ ]:
svd_test = SVD(n_epochs = 40, n_factors = 400, init_std_dev = 0.005, random_state=42)
svd_test.fit(trainset)
predictions = svd_test.test(testset)
# Calculate RMSE
svd_rmse = accuracy.rmse(predictions)

In [ ]:
# Predicted Target Values vs. Actual Target Values
new_df = pd.DataFrame(columns=['uid', 'iid', 'rating'])
i = 0
for (uid, iid, rating) in testset:
    new_df.loc[i] = [uid, iid, rating]
    i = i+1
true = new_df['rating']
pred = []
for i in predictions:
    pred.append(i.est)
fig,axis = plt.subplots(figsize=(8, 5))
sns.boxplot(x=true, y=pred, palette="brg")
plt.title("Predicted Target Values vs. Actual Target Values", fontsize=14)
plt.xlabel("Actual Target Values")
plt.ylabel("Predicted Target Values")
plt.show()

In [ ]:
params = {'n_epochs':40, #[30,40,50],
          'n_factors':400, #[100,200,300],
          'init_std_dev':0.005, #[0.005,0.05,0.1],
          'random_state':[42]} 
metrics = {"RMSE": np.sqrt(mean_squared_error(true, pred))}

experiment.log_parameters(params)
experiment.log_metrics(metrics)
experiment.end()

<a id="conclusion"></a>
## 8. Conclusion
In this project, we succeeded in building an unsupervised machine learning model that is able to recommend movies based on content-based or collaborative filtering and is capable of accurately predicting how a user will rate a movie they have not yet viewed, based on their historical preferences. Our top performing model has a root mean squared error (RMSE) of 0.78, based on a testing set submitted to the EDSA [Kaggle](https://www.kaggle.com/c/edsa-recommender-system-predict/leaderboard) competition.  


The singular value decomposition (SVD) algorithm is a baseline approach to recommender systems, as it has a broad range of applications including dimensionality reduction, solving linear inverse problems, and data fitting. The SVD algorithm generally performs better on large datasets compared to some other models as it decomposes a matrix into constituent arrays of feature vectors corresponding to each row and each column.

<a id="save"></a>
## 9. Save Output

In [ ]:
# Train model on whole dataset
reader = Reader(rating_scale=(train['rating'].min(), train['rating'].max()))
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
svd = SVD(n_epochs = 40, n_factors = 400, init_std_dev = 0.005, random_state=42, verbose=True)
svd.fit(trainset)

# Create Kaggle submission file
predictions = []
for i, row in test.iterrows():
    x = (svd.predict(row.userId, row.movieId))
    pred = x[3]
    predictions.append(pred)
test['Id'] = test['userId'].map(str) +'_'+ test['movieId'].map(str)
results = pd.DataFrame({"Id":test['Id'],"rating": predictions})
results.to_csv("submission.csv", index=False)

In [ ]:
# Save model
model_save_path = "SVD.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(svd,file)

In [ ]:
experiment.display()

<a id="ref"></a>
## 10. References
<a id="ref1"></a>[1] Ansari, A. (n.d.). Internet Recommendation Systems. Retrieved August/September, 2000, from https://www0.gsb.columbia.edu/mygsb/faculty/research/pubfiles/385/Internet Recommendation Systems.pdf

<a id="ref2"></a>[2] Shi, C. (2017, June 27). A Hybrid Recommender with Yelp Challenge. Retrieved from https://nycdatascience.com/blog/student-works/yelp-recommender-part-1/

<a id="ref3"></a>[3] Wikipedia: t-distributed stochastic neighbor embedding. Retrieved from https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding

<a id="ref4"></a>[4] Jolliffe, I.T. & Cadima, J. (2016, April 13). Principal component analysis: a review and recent developments. Retrieved from https://royalsocietypublishing.org/doi/10.1098/rsta.2015.0202

<a id="ref5"></a>[5] Ajitsaria, A. (n.d.). Build a Recommendation Engine With Collaborative Filtering. Retrieved from https://realpython.com/build-recommendation-engine-collaborative-filtering

<a id="ref6"></a>[6] Funk, S. (2006, Deccember 11). Netflix Update: Try This at Home. Retrieved from https://sifter.org/simon/journal/20061211.html

<a id="ref7"></a>[7] Recommender Systems: Content-based Filtering. Retrieved from http://recommender-systems.org/content-based-filtering/